<img src="img/logo.png" width="50%">

# AutoML Vision per la classificazione di radiografie mediche

---

 A cura di: Andrea Crotti e Marco Feroldi

# **Rilevazione della polmonite**


Google Cloud AutoML Vision semplifica la creazione di modelli personalizzati di visione da utilizzare per il riconoscimento di immagini.

Vengono sfruttate architetture di reti neurali per la ricerca al fine di trovare la migliore rete e la configurazione ottimale dei parametri, così da minimizzare la perdita del modello.
Successivamente verrà mostrato come utilizzare Google Cloud AutoML Vision per sviluppare una classificazione di immagini di radiografie mediche del torace, al fine di riuscire a diagnosticare quanto più precisamente possibile la presenza della **polmonite**.

**Polmonite**

<img src="img/pneumonia.jpg" width="100%">

Una normale radiografia (nel pannello di sinistra) del torace mostra polmoni chiari senza aree anomale opache visibili nell’immagine.

La polmonite **batterica**, visibile al centro, presenta invece un consolida- mento lobare focale, in questo caso nel lobo superiore destro, indicato dalle frecce.
Infine la polmonite **virale** (nel pannello di destra) si manifesta con un pat- tern più "interstiziale" più diffuso in entrambi i polmoni.


# **Dataset**
Il dataset contiene:

*   5,232 immagini di radiografie al torace di bambini.
*   3,883 di queste immagini sono esempi di polmonite batterica (2,538) e virale (1,345).
* 1,349 esempi rappresentano immagini di radiografie di bambini sani. 

Il dataset è presente su Kaggle e accessibile gratuitamente dal seguente link:

https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia


# 1. Creare il progetto

Per farlo è necessario andare nella console di Google Cloud, dal seguente link: 

https://cloud.google.com/

<img src="img/console.png" width="100%">

Dopo aver aperto la console, è necessario aprire Cloud ML Vision cliccando prima sulle 3 linee in alto a sinistra nella dashboard. 

Successivamente si seleziona Vision, nella sezione **Intelligenza Artificiale**.

<img src="img/vision.png" width="100%">

Selezionare **Classificazione delle immagini**.

La prima cosa da fare è quella di creare il progetto da utilizzare. 
Per farlo è necessario cliccare in alto a sinistra sul progetto attualmente selezionato (potrebbe essere quello di default).

<img src="img/new-project-click.png" width="100%">

Si clicca poi su **Nuovo progetto**.

<img src="img/new-project.png" width="100%">

Si immette un nome, e si clicca su **Crea**.

<img src="img/new-project-name.png" width="100%">

# 2. Abilitare AutoML Cloud Vision su GCP

Una volta creato il progetto, è necessario fornire le autorizzazioni per l'esecuzione delle API chee AutoML richiede.

Bisogna quindi impostare le API necessarie al progetto, i permessi e il bucket su Cloud Storage per immagazzinare le immagini per la modellizzazione e il test.

Cliccare su **SET UP NOW** nella schermata mostrata.

<img src="img/setup.png" width="100%">

Siamo a questo punto in grado di creare un **Dataset** per costruire un modello di classificazione personalizzato su AutoML.



# 3. Scaricare il Dataset su Google Cloud Storage

Attivare la Cloud shell in alto a destra, per lanciare l’istanza della VM al fine di scaricare il dataset da Kaggle, estrarlo e caricarlo nello storage bucket.

Installare la CLI di Kaggle, al fine di poter scaricare i dataset da Kaggle. Eseguire quindi il seguente codice:

```
pip install kaggle
```

## Shell di breve durata

Risulta importante considerare che l’istanza della Cloud Shell è di breve durata, e non mantiene i cambiamenti effettuati al sistema al termine della sessione: sarà infatti necessario reinstallare pacchetti e riapplicare le configurazioni al riavvio.

Inoltre, se il dataset è particolarmente ampio, esistono altre opzioni, come la possibilità di usare una VM a rotazione, di scaricare il dataset, estrarlo e caricarlo successivamente sul Cloud Storage.

È inoltre presente la possibilità di progettare altre sequenze di operazioni avanzate per immettere i dati in Google Cloud Platform per l’analisi e il machine learning.

## **Kaggle API Token**

Una volta completata l’installazione della CLI di Kaggle, è necessario sca- ricarne il token che permette alla CLI di autenticarsi su Kaggle e scaricare i dataset desiderati.

Per farlo è necessario:
* Accedere al proprio account di Kaggle.
* Andare nella propria area personale.
* Cliccare su **Crea nuovo API Token**.

<img src="img/kaggle_token.png" width="100%">

Una volta scaricato il file *kaggle.json* è necessario caricarlo sulla Cloud Shell. Per farlo è necessario cliccare sui 3 pallini in alto a destra nella shell e selezionare **Carica file**.

<img src="img/kaggle_upload.png" width="100%">

Creare la cartella *.kaggle*, normalmente non esistente

```
mkdir .kaggle
```

Spostare il file caricato nella directory .kaggle, usando il comando seguente:

```
mv kaggle.json .kaggle/kaggle.json
```



## Download ed estrazione del Dataset

Scaricare il dataset da Kaggle e metterlo su Google Storage.

```
kaggle datasets download paultimothymooney/chest-xray-pneumonia
```

Estrarre il dataset, con i due seguenti comandi (uno in sequenza all'altro).

```
unzip chest-xray-pneumonia.zip
unzip chest_xray.zip
```

Spostare il dataset dall'istanza della Cloud Shell (in quanto di breve durata) e spostarla nel bucket di Cloud Storage creato.

```
gsutil -m cp -r chest_xray gs://gs://gcp-prova-vcm//chest_xray/
```

## Dataset snello

È presente un dataset più snello, scaricabile sempre da Kaggle con il seguente comando.

```
kaggle datasets download marcoferoldi/chest-xray-small
```

Estrarre il dataset, con i due seguenti comandi (uno in sequenza all'altro).

```
unzip chest-xray-small.zip
```

Spostare il dataset dall'istanza della Cloud Shell (in quanto di breve durata) e spostarla nel bucket di Cloud Storage creato.

```
gsutil -m cp -r chest_xray_small gs://gs://gcp-prova-vcm//chest_xray/
```


# 3. Preparazione del dataset per la modellizzazione

Avviare il Jupyter Notebook su Google Cloud AI Platform.
Andare quindi nella dashboard di GCP, e nella sezione **Intelligenza Artificiale** selezionare **AI Platform** e poi **Notebooks**.

<img src="img/notebook.png" width="100%">

Creare una nuova istanza di un Notebook, selezionando **Tensorflow 2.0**.

<img src="img/new_istance.png" width="100%">

Scegliere un nome per l'istanza e crearla, eventualmente personalizzandola a piacere.

<img src="img/create_instance.png" width="100%">

Cliccare su **CREA** per creare l'istanza.

Cliccare quindi su **APRI JUPYTERLAB** relativo all'istanza creata.

Prima di costruire un modello di riconoscimento immagini con AutoML Cloud Vision, il dataset va preparato in modo da rispettare alcuni requisiti:

1. Per il training, sono supportate immagini nei formati JPEG, PNG, WEBP, GIF, BMP, TIFF e ICO, che non superino la massima dimensione di 30MB per immagine.

2. I formati JPEG, PNG e GID sono supportati con una massima dimensione per immagine di 1.5MB.

3. È consigliato separare le categorie di immagini creando le sottocartele per categorie, e mettendo le immagini al loro interno.

4. Infine, creare un file CSV che fa riferimento al percorso delle immagini e alle rispettive classi. AutoML utilizza il file CSV per riferirsi elle locazioni delle immagini (e le relative classi) per la fase di training. Il file CSV va posto nello stesso Google Cloud Storage bucket contenente i file delle immagini. Utilizzare il bucked automaticamente creato nella configurazione di AutoML vision. In questo caso, il bucket automaticamente creato si chiama ***gs://ekabasandbox-vcm.***



## 3.1 Script di Pre-processing



In [15]:
import os
import numpy as np
import pandas as pd
from google.cloud import storage

In [16]:
# Instanziare un client
storage_client = storage.Client()

In [17]:
# Inserire il nome del proprio bucket
bucket_name = 'gcp-prova-vcm'

In [4]:
# Prelievo del bucket dal Google Cloud Storage
bucket = storage_client.get_bucket(bucket_name)

In [5]:
# Estrarre i blob nel train dataset
blobs = bucket.list_blobs(prefix='chest_xray/chest_xray/train/')

In [6]:
# Prendere la lista dei blob
blob_list = []
for blob in blobs:
    blob_list.append(blob.name)

In [7]:
# Numero di elementi del dataset

len(blob_list)

5219

In [8]:
print(blob_list[0])
print(blob_list[1])

chest_xray/chest_xray/train/.DS_Store
chest_xray/chest_xray/train/NORMAL/.DS_Store


In [9]:
# Rimuovere i file .DS_Store dalla lista creata dal sistema

i, length = 0, len(blob_list)
bucket_prefix = 'gs://gcp-prova-vcm/'
data = []
for blob in blob_list:
    if '.DS_Store' not in blob:
        entry = [''.join([bucket_prefix, blob]), blob.split('/')[3]]
        data.append(entry)

In [10]:
print(data[0])
print(data[1])

['gs://gcp-prova-vcm/chest_xray/chest_xray/train/NORMAL/IM-0115-0001.jpeg', 'NORMAL']
['gs://gcp-prova-vcm/chest_xray/chest_xray/train/NORMAL/IM-0117-0001.jpeg', 'NORMAL']


In [11]:
# Convertire in un Pandas DataFrame
data_pd = pd.DataFrame(np.array(data))

In [12]:
# Creazione del file CSV
data_pd.to_csv("data.csv", header=None, index=None)

In [13]:
# Upload data.csv sul Google Cloud Storage
output_blob = bucket.blob('data.csv')

In [14]:
output_blob.upload_from_filename('data.csv')

# 4. Creazione del modello con AutoML Vision

In AutoML Vision cliccare su **Nuovo Dataset**

*** IMG ***

Scegliere un nome per il progetto e selezionare il file CSV creato precedentemente e posto nel Cloud Storage bucket creato da AutoML.

Ignorare eventuali messaggi di file duplicati allocati.

Effettuare il **Training** del modello.

*** IMG ***

Dopo aver completato il training, cliccare su **Valutazione** per scoprire le performance del modello creato. 

*** IMG ***

Sono visibili:
* Precision
* Recall
* Confusion Matrix

# 5. Test del modello

Cliccare su **Previsione**

A questo punto è possibile caricare un'immagine e far sì che il modello effettui la sua previsione, comunicando anche le relative percentuali di affidabilità per ciascuna classe di valutazione.